### Goal: Can we do better?
- An issue on the prediction is, the ZRI value is probably not martingale for any time series for a fixed zip code, since there are inflation.
- The idea is that we can try to differentiate the time series, that is, we can try to estimate $Z_i - Z_{i-1}$ instead of $Z_i$.

In [ ]:
import tensorflow as tf;
import numpy as np;
import matplotlib.pyplot as plt;
print(tf.__version__);
import pandas as pd;

from sklearn.preprocessing import StandardScaler;
from sklearn.preprocessing import MinMaxScaler;
from sklearn.metrics import mean_squared_error;

import datetime;

In [ ]:
pd.set_option('display.max_columns', 300);
pd.set_option('display.max_rows', 300);
tf.keras.backend.set_floatx('float64');

In [ ]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

In [ ]:
MONTHS = 60;
SPLIT = 48; # 2015-2018: training, 2019: testing.
# BATCH_SIZE = 19; # used in NN_v1
BATCH_SIZE = 24;
WINDOW_SIZE = 3;

TEST_LENGTH = MONTHS - SPLIT;

### Data preparation

In [ ]:
multi_data = pd.read_csv('../data/full_dataset_engineered.csv', index_col=0);
zip_ids = multi_data.index.unique();

multi_data.drop(["City", "State", "Metro", "CountyName", "year", "month", "year-month"],\
                 axis = 1, inplace = True);

multi_data.head()

In [ ]:
# from google.colab import drive 
# drive.mount('/content/gdrive')

# multi_data = pd.read_csv('gdrive/My Drive/full_dataset_unscaled.csv', index_col=0);
# zip_ids = multi_data.index.unique();

# multi_data.drop(["City", "State", "Metro", "CountyName", "year", "month", "datetime"],\
#                  axis = 1, inplace = True);

# multi_data.head()

In [ ]:
FEATURES = multi_data.shape[1] - 1;

feature_name = list(multi_data.columns);

In [ ]:
# In our first try, just look at the zip codes in NY. zip:10001-14905
multi_zip = zip_ids;
# list(multi_data[(multi_data.index >= 10001) & (multi_data.index <= 14905)].index.unique());
print(len(multi_zip))

### Utility functions

In [ ]:
@tf.autograph.experimental.do_not_convert
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series); #(43,)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True);
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1)); #(13,43)
    dataset = dataset.shuffle(shuffle_buffer)\
                     .map(lambda window: (window[:-1], window[-1][0]));
    dataset = dataset.batch(batch_size);
    return dataset;

In [ ]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time Frame")
    plt.ylabel("ZRI")
    plt.grid(True)

### Neural network center

In [ ]:
def NN_forecast(model, series_transformed, pure=True):
    forecast = []
    results = []
    for time in range(MONTHS - WINDOW_SIZE):
        forecast.append(model.predict(series_transformed[np.newaxis, time:time + WINDOW_SIZE, :]))

    results = [float(x[-1][0]) for x in forecast];
    actual = list(series_transformed[WINDOW_SIZE:, 0]);
    
    if not pure:
        return results, actual;

    timeline = range(WINDOW_SIZE, MONTHS);
    time_test = range(SPLIT, MONTHS);
    forecast = series_transformed[SPLIT - WINDOW_SIZE:,:].copy();

    for time in range(TEST_LENGTH): # Change temp[time + WINDOW_SIZE]
        forecast[time + WINDOW_SIZE, 0] =\
        model.predict(forecast[np.newaxis, time:time + WINDOW_SIZE, :])[-1][0];

    pure_forecast = forecast[WINDOW_SIZE:,0];
    
    print(len(results), len(actual), len(pure_forecast))
    
    return results, actual, pure_forecast;

### Load model

In [ ]:
model = tf.keras.models.load_model('./saved_models/ZRI_and_all.h5');

In [ ]:
collection_importance = [];

for zip_num in multi_zip:
    scaler = MinMaxScaler();
    series = np.array(multi_data[multi_data.index == zip_num]);
    series_transformed = scaler.fit_transform(series);

    # Forecasting
    results, actual = NN_forecast(model, series_transformed, pure=False);

    # Compute MSE
    mse = mean_squared_error(actual[:-TEST_LENGTH], results[:-TEST_LENGTH])**0.5;
    # Usually we need to multiply by scaler.data_range_[0], but can ignore this.
    TRIALS = 1;
    PLOT_IMPORTANCE = True;
    
    perm_importance = [zip_num];
    for i in range(len(feature_name)):
        mse_feat = [];
        for _ in range(TRIALS):
            seq_perm = series_transformed.copy();
            seq_perm[:, i] = np.random.permutation(seq_perm[:,i])
            results, actual = NN_forecast(model, seq_perm, pure = False);
            mse_feat.append(mean_squared_error(actual[:-TEST_LENGTH], results[:-TEST_LENGTH])**0.5);

        perm_importance.append(sum(mse_feat)/TRIALS/mse - 1.0);
        # print("{}, {:.4f}".format(feature_name[i], sum(mse_feat)/TRIALS/mse - 1.0));

    collection_importance.append(perm_importance);


In [ ]:
importance_df = pd.DataFrame(collection_importance, columns = ["zip"] + feature_name);
importance_df.to_csv("./Results in CSV files/Importance.csv", index = False);

In [ ]:
importance_df